<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/node_postprocessor/CohereRerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jina Rerank

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-postprocessor-jinaai-rerank

In [ ]:
!pip install llama-index

In [3]:
import os
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.core.response.pprint_utils import pprint_response

Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [5]:
from llama_index.embeddings.jinaai import JinaEmbedding
api_key = os.environ["JINA_API_KEY"]
jina_embeddings = JinaEmbedding(api_key=api_key)

# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

# build index
index = VectorStoreIndex.from_documents(documents=documents, embed_model=jina_embeddings)

#### Retrieve top 10 most relevant nodes, then rerank with Jina Rerank

In [6]:
import os
from llama_index.postprocessor.jinaai_rerank import JinaRerank


jina_rerank = JinaRerank(api_key=api_key, top_n=2)

In [15]:
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[jina_rerank]
)
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

In [16]:
pprint_response(response)

Final Response: Sam Altman was asked to become the president of Y
Combinator (YC) after the original founders decided to step down and
reorganize the company to ensure its longevity. Initially hesitant due
to wanting to start a startup focused on nuclear reactors, Sam
eventually agreed to take over as president starting with the winter
2014 batch.
